<a href="https://colab.research.google.com/github/Kensuzuki95/AIF360/blob/master/Web_Scraping_Store_Info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup

url = 'https://shop.tomods.jp/all?keyword=%E3%83%88%E3%83%A2%E3%82%BA'

# Send a GET request to the URL
response = requests.get(url)

# Create a BeautifulSoup object from the response text
soup = BeautifulSoup(response.text, 'html.parser')

# Find all store name elements
store_name_elements = soup.find_all('button', class_='font-bold ml-1.5 text-left text-[14px] hover:opacity-50 transition-opacity')

# Extract and print the store names
for store_name_element in store_name_elements:
    store_name = store_name_element.text.strip()
    print(store_name)

薬局トモズ 成城コルティ店
トモズ 白金高輪店
トモズ アークヒルズ店
トモズ 青葉台店
トモズ 青葉台東急スクエア店
トモズ 赤坂店
トモズ 秋葉原店
トモズ アコルデ代々木上原店
トモズ 麻布十番店
薬局トモズ アスタ田無店
トモズ アトレヴィ田端店
トモズ アトレ浦和店
トモズ アトレ川崎店
トモズ アトレ吉祥寺店
トモズアトレ四谷店
トモズ 有明ガーデン店
トモズ イクスピアリ店
トモズ 池尻大橋店
トモズ 磯子店
トモズ 稲田堤店
薬局トモズ 今井南町店
トモズ 梅里店
薬局トモズ 同潤会松江店
トモズ ＥＱＵＩＡ朝霞
トモズ EQUIA 曳舟店
トモズエキュートエディション新橋店
トモズ エスパル仙台店
トモズエトモ祐天寺店
トモズ 恵比寿 ガーデンプレイス店
トモズ Emio桜台店
トモズ Emio東久留米店
トモズ 王子店
薬局トモズ 大泉学園店
トモズ 大泉学園店
薬局トモズ 大倉山店
トモズ 大倉山店
トモズ 大手町カンファレンスセンター店
トモズ 大手町プレイス店
トモズ 大宮店
トモズ 大谷口店
薬局トモズ 大山店
トモズ 大山店
薬局トモズ 押上店
トモズ小田急マルシェ祖師ヶ谷大蔵店
トモズ 小田急マルシェ玉川学園前店
トモズ お花茶屋店
トモズ 学芸大学店
薬局トモズ 葛西店
薬局トモズ 葛飾青戸店
トモズ カトレヤプラザ伊勢佐木店
薬局トモズ 蒲田店
トモズ 蒲田店
トモズ 上板橋北口店
薬局トモズ 上板橋南口店
トモズ 上板橋南口店
トモズ 上馬店
トモズ 上野毛店
トモズ 神谷町店
トモズ 亀有駅北店
薬局トモズ 亀戸東口店
トモズ 茅場町店
トモズ 川口店
薬局トモズ 神田和泉町店
薬局トモズ 関東労災病院前店
トモズ菊名店
トモズ 北浦和店
トモズ 北浦和東口店
トモズ キテミテマツド店
トモズ 清瀬店
トモズ 銀座三丁目店
薬局トモズ 銀座八丁目店
トモズ クイーンズ伊勢丹笹塚店
トモズ クイーンズ伊勢丹石神井公園店
トモズ クイーンズ伊勢丹杉並桃井店
トモズ クイーンズ伊勢丹仙川店
トモズ クイーンズスクエア横浜店
薬局トモズ椚田店
トモズ 久米川店
トモズ グリーンマークシティ 松戸新田店
トモズ ゲートシティ大崎店
薬局トモズ 京王リトナード永福町店
トモズ 京王リトナード永福町店
トモズ 京王リトナード つつじヶ丘店
